In [ ]:
%load_ext autoreload
%autoreload 2

# Instamatic

Instamatic is a tool for automated electron diffraction data collection. It has interfaces for interfacing with the TEM (JEOL/TFS) and several cameras (Gatan/ASI Timepix/TVIPS).

https://github.com/stefsmeets/instamatic

This notebook shows how to process a grid montage using `instamatic`, pick holes, and set up an acquisition (`acquire_at_items`).

In [ ]:
from instamatic.montage import *
import numpy as np
np.set_printoptions(suppress=True)

## Setting up the montage

Load the `gm.mrc` file and the associated images. For SerialEM data, the gridshape must be specified, because it cannot be obtained from the data or `.mdoc` direction.

In [ ]:
m = Montage.from_serialem_mrc('C:\\s\\2020-02-05\\lta_crystals\\gm.mrc', 
                              gridshape=(5,5))
m.gridspec

First, we can check what the data actually look like. To do so, we can simply `stitch` and `plot` the data using a `binning=4` to conserve a bit of memory. This naively plots the data at the expected positions. Although the stitching is not that great, it's enough to get a feeling for the data.

Note that SerialEM includes the pixel coordinates in the `.mdoc` file, so it is not necessary to calculate these again. Instead, the `PieceCoordinates` are mapped to `m.coords`.

In [ ]:
# Use `optimized = False` to prevent using the aligned piece coordinates
m.stitch(binning=4, optimized=False)
m.plot()

SerialEM has also already calculated the aligned image coordinates (`AlignedPieceCoordsVS`/`AlignedPieceCoords`). These can be accessed via the `.optimized_coords` attribute. To plot, you can do the following:

In [ ]:
# optimized = True is the default, so it can be left out
m.stitch(binning=4, optimized=True)
m.plot()

It is still possible to try to get better stitching using the algorithm in `instamatic`
 1. Better estimate the difference vectors between each tile using cross correlation
 2. Optimize the coordinates of the difference vectors using least-squares minimization

In [ ]:
# Use cross correlation to get difference vectors
m.calculate_difference_vectors(threshold='auto', 
                               verbose=False, 
                               segment=False,
                               method='imreg', 
                               plot=False)

# plot the fft_scores
m.plot_fft_scores()

# plot the pixel shifts
m.plot_shifts()

# get coords optimized using cross correlation
m.optimize_montage_coords(plot=True)

# stitch image, use binning 4 for speed-up and memory conservation
m.stitch(binning=4)

# plot the stitched image
m.plot()

When the image has been stitched (with or without optimization), we can look for the positions of the grid squares/squircles. To do so, call the method `.find_holes`. `Instamatic` does not know the imaging conditions, so we must first set the `.mode` attribute. It already knows the magnification and will then read out the stagematrix from the config files.

In [ ]:
m.mode = 'lowmag'

m.set_calibration('lowmag', 100)

stagecoords, imagecoords = m.find_holes(plot=True, diameter=45e3, tolerance=0.1)
stagecoords = stagecoords.astype(int)

It is possible to optimize the stage coordinates for more efficient navigation. In this example, the total stage movement can be reduced by about 75%, which will save a lot of time. The function uses the _two-opt_ algorithm for finding the shortest path: https://en.wikipedia.org/wiki/2-opt.

In [ ]:
from instamatic.navigation import sort_nav_items_by_shortest_path
stagecoords = sort_nav_items_by_shortest_path(stagecoords, plot=True);

The `stagecoords` can be used to set up an automated **acquire at items**. First, initialize the `ctrl` object from `instamatic`.

In [ ]:
from instamatic import TEMController
ctrl = TEMController.initialize()

Next, we should set up an acquisition function for each stage position. This should:

1. Center the grid square by aligning it with a reference image
2. Take an image at high mag
3. Store the image and the corresponding stage position in a buffer

To do so, we much first obtain a reference image from a grid square. The magnification should be so that the grid square fits in the view of the image. In this example, we use `300x` in `mag1`.

In [ ]:
# set microscope conditions
ctrl.mode = 'lowmag'
ctrl.magnification.value = 300
binsize = 2

# reference image of a centered grid square
ref_img = ctrl.getRawImage()

buffer = []
stagepos = []


def acquire_func(ctrl):
    # Align to template
    ctrl.align_to(ref_img, apply=True)
    
    # obtain image
    img, h = ctrl.getImage(binsize=binsize)  
    buffer.append(img)
    
    # store stage position and image somewhere
    pos = ctrl.stage.get()
    stagepos.append(pos)

When the function is defined, we can pass it and the list of grid square stage coordinates to the function `ctrl.acquire_at_items`, which will automate the function at each stage position.

In [ ]:
sel = stagecoords[0:20]  # Acquire at the first 10 items
ctrl.acquire_at_items(sel, acquire=acquire_func)

Here is a minimal example of how the acquire functions can be changed to collect data can be saved to a `.nav` file which can be read by `SerialEM`. 

This makes use of the ability to pass a `post_acquire` function to `.acquire_at_items`. The post acquisition can be used to save the images as well as the required metadata to `SerialEM` format, making use of the `instamatic.serialem` module.

In [ ]:
import mrcfile
from instamatic.serialem import MapItem, write_nav_file

# reference image of a centered grid square
ref_img = ctrl.getRawImage()

# empty buffers
buffer = []
stagepos = []

   
def write_mrc_stack(fn:str, data: list, overwrite: bool=True, mmap:bool = True):
    """Write a stack of images to an mrc file."""
    if mmap:
        shape = (len(buffer), *buffer[0].shape)
        with mrcfile.new_mmap(fn, shape=shape, overwrite=True, mrc_mode=1) as f:
            for i, im in enumerate(buffer):
                f.data[i] = im
    else:
        data = np.array(data)
        # mrc can only be saved as a 16-bit integer
        data = data.astype(np.int16)
        try:
            f = mrcfile.new(fn, data=data, overwrite=overwrite)
        except OSError:
            f.close()
    

    def post_acquire(ctrl):
    fn_nav = 'instamatic.nav'
    fn_mrc = 'mmm.mrc'
    
    write_mrc_stack(fn_mrc, buffer)
    
    items = []
    
    magnification = ctrl.magnification.value
    mode = ctrl.mode
    mapscalemat = getattr(config.calibration, f'stagematrix_{mode}')[magnification]
    mapscalemat = [item/binsize for item in mapscalemat]
    
    for i, image in enumerate(buffer):
        x, y, z, _, _ = stagepos[i]
        shape = image.shape
        # binsize = ctrl.cam.getBinning()[0]

        d = {}
        d['StageXYZ'] = x / 1000, y / 1000, z / 1000
        d['MapFile'] = fn_mrc
        d['MapSection'] = i
        d['MapBinning'] = binsize
        d['MapMagInd'] = ctrl.magnification.absolute_index + 1  # SerialEM is 1-based
        d['MapScaleMat'] = mapscalemat
        d['MapWidthHeight'] = shape

        map_item = MapItem.from_dict(d)
        items.append(map_item)

    write_nav_file(fn_nav, *items)
    
    print(f"Data saved to `{fn_nav}` and `{fn_mrc}` ({len(buffer)} images)")

Next we call `ctrl.acquire_at_items` as before with the new `post_acquire` function.

In [ ]:
sel = stagecoords[0:20]  # Acquire at the first 10 items
ctrl.acquire_at_items(sel, 
                      acquire=acquire_func, 
                      post_acquire=post_acquire)

Now load up the `instamatic.nav` file in `SerialEM` to see the result!